# imdb_movie_scraping

The dataset is a webscraped from IMDB website using BeautifulSoup. 

In [2]:
from requests import get #the package which fetches the HTML doc from the url for us
import re
import pandas as pd
import sqlite3

Say if you want to scrape a list of 1000 movies, we would have to send in 1000 requests to the website. Assuming each request takes 1 second to execute, it would take a 1000 seconds to execute. When we explore the website a bit we find noval ways which will help us to execute our scraping much more faster. 

In [2]:
url = "http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1"

# Inspecting the link

Exploring the IMDB site for a while we find that while using advanced search feature we can have look at the best movies in the given time frame by 50 movies per page. This reduces our time 50 times as we can extract 50 movies per request.

further let's explore the url to have a better understanding of waht is happeninng on each request. The link has following elements:

- *release_date*: this is takes in the value for the year we are interested in. (2017 in our case)
- *sort*: this takes in the value by which we want to sort our list. (num_votes,desc in our case, desc suggests descending order)
- *page*: this takes the page number we are interested in. (1 in our case)

Further when you click on the next tab, we get an additional element in the link:
"http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=2&ref_=adv_nxt"

- ref: this suggests if we want to go on the next or the previous page.

In [3]:
response = get(url)
print(response.text[:500]) #acccessing the .text attribute of response





<!DOCTYPE html>
<html
xmlns:og="http://ogp.me/ns#"
xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">
            <script type="text/javascript">var ue_t0=window.ue_t0||+new Date();</script>
            <script type="text/javascript">
                var ue_mid = "A1EVAM02EL8SFB"; 
                var


# Understanding the HTML structure of the page

As we can see get() method pulls up the html document for the url. To further pull out the data we want we need to closely inspect the HTML content of the page. We can turn to the developer tools of chrome for this, cntrl+shift+i or simply right clicking on the element you want to study and seelcting the inspect element will pull up that data set for you. (This is performed on chrome bust should work with other browsers as well.)

We observe that there is a div tag for each movie. Thus, we can simply loop through the 50 dev tags assosicated with each movie using BeautifulSoup.

In [4]:
from bs4 import BeautifulSoup as bs # importing BeautifulSoup as bs

html_soup = bs(response.text, "html.parser") #using python's built in HTML parser
type(html_soup)

bs4.BeautifulSoup

Before extracting 50 div containers, we need to figure out what makes them unique. While exploring the HTML content we find that the class attribute has two values "lister-item" and "mode-advanced". This combination is unique to all the div containers. 

In [5]:
movie_containers = html_soup.find_all("div", class_ = "lister-item mode-advanced") #find_all() used to find all the tags
print(type(movie_containers))
len(movie_containers)

<class 'bs4.element.ResultSet'>


50

find_all() returned a ResultSet object with the length of 50 movies we are interested in.

Now, we'll select the conatainers one by one and extract the elements of our interest like:

- The name of movie.
- The year of release.
- The IMDB rating.
- The metascore.
- The number of votes
etc...

In [19]:
#list to store scraped value data in:
movie_names = []
year_release = []
imdb_ratings = []
metascores = []
votes = []
movie_description = []
certificate = []
runtime = []
genre = []
director_name = []
star_cast = []
gross_value = []

#extract data from individual movie container
for container in movie_containers:
    
   
    #if movie has Metascore, then extract:
    if container.find("div", class_ = "ratings-metascore") is not None:
        
                #the movie_name
                name = str(container.h3.a.text)
                #The year of release
                release = container.find("span", class_ = "lister-item-year text-muted unbold").text
                #The ratings for the movie
                ratings = float(container.strong.text)
                #metascore for the movie
                meta = container.find("span", class_ = "metascore").text
                #The number of votes
                vote = container.find("span", attrs = {"name":"nv"})['data-value']
                #the certificate for the movie
                if container.find("span", class_ = "certificate") is not None:
                    certi = container.find("span", class_ = "certificate").text
                else:
                    certi = None
                 #The runtime for the movie   
                if container.find("span", class_ ="runtime") is not None:
                    run = container.find("span", class_ ="runtime").text
                else:
                    run = None
                 #The genre of the movie   
                if container.find("span", class_ ="genre") is not None:
                    gen = container.find("span", class_ ="genre").text
                else:
                    gen = None   
                
                #fetching all <p> tags
                content = container.find_all("p")
                
                #The movie description
                if content[1] is not None:
                    desc = content[1].text
                else:
                    desc = None
                
                #subsetting all the <a> tags in 3rd <p> tag
                content_2 = content[2].find_all("a")
                
                #the director
                if content_2[0] is not None:
                    director = content_2[0].text
                else:
                    director = None
                
                #the gross value
                if len(container.find_all("span", attrs = {"name":"nv"})) >= 2:
                    gross = container.find_all("span", attrs = {"name":"nv"})[1]['data-value']
                else:
                    gross = None
                    
                 #extracting artists names
                if content_2[1] is not None:
                    temp = []
                    for i in range(len(content_2)-1):
                        temp.append(content_2[i].text)
                else:
                    for i in range(len(content_2)-1):
                        temp.append(None)
                        
                #Cleaning the data using regular Expressions:
                
                name = str(name)
                desc = str(re.findall(r"[^\\r\n].+",desc))
                gen = re.findall(r"([^\\r\\n\s,][a-zA-Z]+)",gen)
                
                if (gross != None):
                    gross = int(gross.replace(",",""))
                
                if (run != None):
                    run = int(re.findall(r"[0-9].+[^a-zA-Z-]", run)[0])
                    
                if (release != None):
                    release = str(re.findall(r"[0-9].+[^a-zA-Z-]",release))
                    
                for i in range(len(content_2)-1):
                    for j in range(len(gen)):
                        movie_names.append(str(name))
                        year_release.append(release)
                        imdb_ratings.append(ratings)
                        metascores.append(int(meta))
                        votes.append(int(vote))
                        movie_description.append(desc)
                        gross_value.append(gross)
                        runtime.append(run)
                        certificate.append(str(certi))
                        director_name.append(str(director))
                        star_cast.append(str(temp[i]))
                        genre.append(str(gen[j]))
        
    
        
                

In [20]:
import pandas as pd

test_df = pd.DataFrame({"movie_names":movie_names,
                        "year_release":year_release,
                        "imdb_ratings":imdb_ratings,
                        "metscores":metascores,
                        "votes":votes,
                        "movie_description":movie_description,
                        "certificate":certificate,
                        "runtime":runtime,
                        "genre":genre,
                        "director_name": director_name,
                        "star_cast": star_cast,
                        "gross_value":gross_value
                       })

In [39]:
test_df.head(2)

,certificate,director_name,genre,gross_value,imdb_ratings,metscores,movie_description,movie_names,runtime,star_cast,votes,year_release
0,R,James Mangold,Action,226277068.0,8.1,77,"[""In the near future, a weary Logan cares for ...",Logan,137,James Mangold,452557,['2017)']
1,R,James Mangold,Drama,226277068.0,8.1,77,"[""In the near future, a weary Logan cares for ...",Logan,137,James Mangold,452557,['2017)']


## Everything went just as expected!

As a side note if you run the code in a country where english is not the main language, it is very likely that you will get the movie names translated into the main language of that country. To avoid such issues, include the headers = {"Accept-Language": "en-US, en;q=0.5"} as an argument in the get() command.

# Script for multiple pages

Building a script to scrape multiple pages can be a bit more challenging, we will have to build upon our old script by adding three more things:

- Making all the requests we want from within the loop.
- Controlling the loops rate to avoid bombarding the server with requests.
- Monitoring the loop while it is in progress.

We'll scope through the first 4 pages of each year in the range 2000-2018

# Changing URL parameters

As described before, the URl changes certain logic as the web page changes.

As we are making requests, we'll only have to vary the values of two parameters of the URL: "release_date" and "page".

In [23]:
pages = [str(i) for i in range(1,11)] #creating list of strings corresponding to 4 pages
years = [str(i) for i in range(2017,2019)] #creating list corresponding to years 2000-2018

# Controlling the crawl rate

If we avoid flooding the server with tens of request per second, then we are much likely to avoid our Ip being banned permenantly. We also avpid disrupting the activity of the website we scrape by allowing server to respond to other user's requests too.

We'll control the loop's rate by using the sleep() function in the python's "time" module. This will pause the execution of the loop for a specified amount of seconds.

To mimic the human behavious and to render our requests legit we will vary the amount of waiting time between requests by using the randint() function from python's "random" module



In [24]:
from time import sleep
from random import randint

# Monitoring the loop as it's still going

Given that we have so many pages to scan through, it's better to have a way to moniter them while we are looping through them. This process in completely optional but is very helpful while debugging the process.If you are looping through say a 100+ pages, I'd say this is a must have feature.

For our script, we'll make sure to use this feature and measure the following parameters:

- The frequency of requests, just to make sure we are not overloading the server.

- The number of requests, so we can halt the loop incase the number of requests is exceeded.

- The status code of our requests, so we make sure the server is sending back the correct responses.

In [26]:
from IPython.core.display import clear_output
from time import time

#redeclaring the variables

movie_names = []
year_release = []
imdb_ratings = []
metascores = []
votes = []
movie_description = []
certificate = []
runtime = []
genre = []
director_name = []
star_cast = []
gross_value = []

#preparing the moniter of the loop
start_time = time()
requests = 0


    
#for every year in the interval 2000-2018
for year in years:

    #for every page in the onterval 1-4
    for page in pages:

        #make a get request
        response = get("http://www.imdb.com/search/title?release_date=" + year + "&sort=num_votes,desc&page=" + page)

        #pause the loop
        sleep(randint(8,15))

        #monitor the requests
        requests += 1
        sleep(randint(1,3))
        elapsed_time = time() - start_time
        print("Request: {}, Frequency: {} requests/s".format(requests, requests/elapsed_time))
        print(page, year)
        clear_output(wait = True)

        #throw a warning for non-200 status codes
        if response.status_code != 200:
            warn("Request: {}, Status Code: {} ".format(requests, response.status_code))


        #break the loop if the frequency of request is too higih
        if requests > 200:
            warn("Number of requests was greater than expected.")
            break

        #parse the content through the html.parser using BeautifulSoup
        html_page = bs(response.text, "html.parser")

        #select all 50 movie container for a single page
        containers = html_page.find_all("div", class_ = "lister-item mode-advanced")

        #for every movie of the 50 movies
        for container in containers:

            #if movie has Metascore, then extract:
            if container.find("div", class_ = "ratings-metascore") is not None:

                #the movie_name
                name = str(container.h3.a.text)
                #The year of release
                release = container.find("span", class_ = "lister-item-year text-muted unbold").text
                #The ratings for the movie
                ratings = float(container.strong.text)
                #metascore for the movie
                meta = container.find("span", class_ = "metascore").text
                #The number of votes
                vote = container.find("span", attrs = {"name":"nv"})['data-value']
                #the certificate for the movie
                if container.find("span", class_ = "certificate") is not None:
                    certi = container.find("span", class_ = "certificate").text
                else:
                    certi = None
                 #The runtime for the movie   
                if container.find("span", class_ ="runtime") is not None:
                    run = container.find("span", class_ ="runtime").text
                else:
                    run = None
                 #The genre of the movie   
                if container.find("span", class_ ="genre") is not None:
                    gen = container.find("span", class_ ="genre").text
                else:
                    gen = None   
                
                #fetching all <p> tags
                content = container.find_all("p")
                
                #The movie description
                if content[1] is not None:
                    desc = content[1].text
                else:
                    desc = None
                
                #subsetting all the <a> tags in 3rd <p> tag
                content_2 = content[2].find_all("a")
                
                #the director
                if content_2[0] is not None:
                    director = content_2[0].text
                else:
                    director = None
                
                #the gross value
                if len(container.find_all("span", attrs = {"name":"nv"})) >= 2:
                    gross = container.find_all("span", attrs = {"name":"nv"})[1]['data-value']
                else:
                    gross = None
                    
                 #extracting artists names
                if content_2[1] is not None:
                    temp = []
                    for i in range(len(content_2)-1):
                        temp.append(content_2[i].text)
                else:
                    for i in range(len(content_2)-1):
                        temp.append(None)
                        
                #Cleaning the data using regular Expressions:
                
                name = str(name)
                desc = str(re.findall(r"[^\\r\n].+",desc))
                #temp = re.findall(r"'([^']*)'",temp)
                gen = re.findall(r"([^\\r\\n\s,][a-zA-Z]+)",gen)
                
                if (gross != None):
                    gross = int(gross.replace(",",""))
                
                if (run != None):
                    run = int(re.findall(r"[0-9].+[^a-zA-Z-]", run)[0])
                    
                if (release != None):
                    release = str(re.findall(r"[0-9].+[^a-zA-Z-]",release))
                 
                #storing data in the list objects
                for i in range(len(content_2)-1):
                    for j in range(len(gen)):
                        movie_names.append(str(name))
                        year_release.append(release)
                        imdb_ratings.append(ratings)
                        metascores.append(int(meta))
                        votes.append(int(vote))
                        movie_description.append(desc)
                        gross_value.append(gross)
                        runtime.append(run)
                        certificate.append(str(certi))
                        director_name.append(str(director))
                        star_cast.append(str(temp[i]))
                        genre.append(str(gen[j]))

Request: 20, Frequency: 0.06919363152835704 requests/s
10 2018


In [41]:
#storing scraped data into a data frame.

imdb_movie_dataset = pd.DataFrame({"movie_names":movie_names,
                        "year_release":year_release,
                        "imdb_ratings":imdb_ratings,
                        "metscores":metascores,
                        "votes":votes,
                        "movie_description":movie_description,
                        "certificate":certificate,
                        "runtime":runtime,
                        "genre":genre,
                        "director_name": director_name,
                        "star_cast": star_cast,
                        "gross_value":gross_value
                       })

imdb_movie_dataset.head(3)

,certificate,director_name,genre,gross_value,imdb_ratings,metscores,movie_description,movie_names,runtime,star_cast,votes,year_release
0,R,James Mangold,Action,226277068.0,8.1,77,"[""In the near future, a weary Logan cares for ...",Logan,137,James Mangold,452598,['2017)']
1,R,James Mangold,Drama,226277068.0,8.1,77,"[""In the near future, a weary Logan cares for ...",Logan,137,James Mangold,452598,['2017)']
2,R,James Mangold,Sci,226277068.0,8.1,77,"[""In the near future, a weary Logan cares for ...",Logan,137,James Mangold,452598,['2017)']


In [30]:
imdb_movie_dataset.to_csv("imdb_movie_dataset.csv", encoding = 'UTF-8') #stores the DataFrame as a csv file

In [33]:
imdb_movie_dataset.shape # checking the shape of the cleaned dataset.

(2887, 12)

In [36]:
#extracting movie names for further scraping
movies = imdb_movie_dataset['movie_names']
movies.values

array(['Logan', 'Logan', 'Logan', ..., 'Zhuo yao ji 2', 'Zhuo yao ji 2',
       'Zhuo yao ji 2'], dtype=object)

In [37]:
movies.to_csv("movies.csv", encoding = 'UTF-8') #saving the names of movies as a csv file

# Accessing facebook data using facebook API



In [1]:
#calling dependencies
import numpy as np
import pandas as pd
import facebook
import requests
import json
import time

## Facebook API Access

Facebook implements OAuth 2.0 as its standard authentication mechanism. You need to get an _access token_ by logging in to your Facebook account and go to https://developers.facebook.com/tools/explorer/ to obtain an ACCESS_TOKEN. 
See [http://facebook-sdk.readthedocs.io/en/latest/api.html](http://facebook-sdk.readthedocs.io/en/latest/api.html)

In [1]:
app_id = """Type your app_id here"""
app_secret = """Type your app_secret here"""

# the following are obtained from the app that you create using your FB developer account

Since the facebook API token is just valid for 2 hour we defined a function which we call from within our for loop to re genereate the access token

In [4]:
def get_fb_token(app_id, app_secret):           
    payload = {'grant_type': 'client_credentials', 'client_id': app_id, 'client_secret': app_secret}
    file = requests.post('https://graph.facebook.com/oauth/access_token?', params = payload)
    result = file.json()
    rtr = result['access_token']
    return rtr

In [6]:
movies_dataset = pd.read_csv("imdb_movie_dataset.csv")
movies_dataset.head(2)
movies = movies_dataset["movie_names"].unique()

# Running the query to scrape Facebook on AWS:

The code below scrapes Facebook for the posts by a particular movie page. This data will further be used for developing tags and to perform text analytics. I ran the project on AWS beacaus we wanted to get the data over a period of time.

In [ ]:
from IPython.core.display import clear_output

cycle = 1
count = 0
post_id = []
post_message = []
post_shares = []
post_likes = []
post_comments = []
post_comment_count = []
post_created_time = []
comments_id = []
comments_created_time = []
post_reference_id = []
post_movie = []


for movie in movies:
    
    if (count==100) is True:
        
        print('sleep cycle: ',cycle)
        
        
        post_data_fb = pd.DataFrame({'post_id':post_id,
                            'post_movie': post_movie,
                            'created_at':post_created_time,
                            'post_message':post_message,
                            'post_comment_count':post_comment_count,
                            'post_likes_count':post_likes,
                            'post_shares_count':post_shares})
                            
        
        post_data_fb.to_csv("post_data_fb.csv", encoding='UTF-8')                    
        
        
        comments_posts_fb = pd.DataFrame({'created_at': comments_created_time,
                                 'comments_id':comments_id,
                                 'post_comments': post_comments,
                                 'post_id':post_reference_id})
                                 
        
        comments_posts_fb.to_csv("comments_posts_fb.csv", encoding = 'UTF-8')                         
        
        
        time.sleep(3600)
        count = 0
        ACCESS_TOKEN = get_fb_token(app_id,app_secret)
        g = facebook.GraphAPI(ACCESS_TOKEN, version = '2.7')
        cycle += 1
        

    
    pages = g.request('/search?q='+movie+'&type=page&limit=1')
    page_id = pages['data'][0]['id']
    all_fields = 'id,created_time,message,comments.summary(true),likes.summary(true),shares'
    posts = g.get_connections(page_id,'posts',fields=all_fields)
    #posts = requests.get(posts['paging']['next']).json()
    count += 1
    print('Request number: ', count)
    clear_output(wait=True)
    
    
    for post in posts['data']:
        if 'comments' in post:
            #extracting comments count
            post_comment_count.append(post['comments']['summary']['total_count'])
        else:
            post_comment_count.append("NA")

        if 'id' in post:
            #extracting post id
            post_id.append(post['id'])
            post_movie.append(movie)
        else:
            post_id.append("NA")
            
        if 'created_time' in post:   
            #extracting Created Time
            post_created_time.append(post['created_time'])
        else:
            post_created_time.append("NA")

        if 'likes' in post:
            #extracting likes
            post_likes.append(post['likes']['summary']['total_count'])
        else:
            post_likes.append("NA")
            
        if 'shares' in post:    
            #extracting share count
            post_shares.append(post['shares']['count'])
        else:
            post_shares.append("NA")
            
        if 'message' in post:
            #extracting message
            post_message.append(post['message'])
        else:
            post_message.append("NA")

        if (posts['data'][0]['comments']['data'] == []) is not True:
            
            for i in range(len(post['comments']['data'])):
                #extracting comments created_time, id, message
                comments_created_time.append(post['comments']['data'][i]['created_time'])
                comments_id.append(post['comments']['data'][i]['id'])
                post_comments.append(post['comments']['data'][i]['message'])
                post_reference_id.append(post['id'])

# Using API for twitter:

Using the APIs for twitter we extracted the tweets user_ids and the rewtweet counts for analysis and tag creation.

In [109]:
# Installing dependencies
import pandas as pd
import numpy as np
import json
import nltk
from textblob import TextBlob as tb
from nltk.corpus import stopwords
import twitter

In [110]:
# Creating the twitter connection to API
from twitter import *
auth = twitter.oauth.OAuth('Consumer_key', 'consumer_secret','oauth_token', 'oauth_token secret')
twitter_api = twitter.Twitter(auth=auth)

In [ ]:
count =1
doc=''
sep = ':'
tweets=pd.DataFrame(columns=['movie_name','user_id','tweet_id','tweet_text','hashtags','user_mentions','created_date','retweet_count','screen_name'])
movie_tweets=pd.DataFrame(columns=['movie_name','tweets'])
i=1
movie_count=1
for movie in movies:
    doc=''
    if count>=100:
        time.sleep(60*15)
        count=1
        tweets.to_csv("Tweets.csv",encoding='utf-8')
        movie_tweets.to_csv("Movie_Tweets.csv",encoding='utf-8')
        auth = twitter.oauth.OAuth('962724602685394944-eiWoSq1e4P8ATMIm16KMw4SPA2jUrKF', 'FUMWZc2P2B5fJ9M0HXUcMMMifGyODyXdUFipgjTjruY2J',
                           '9qQwgXZjo0rYbfa1suN0bOiqD', '64GmL60QvLGm2IYDnCw4WMwnFaGzbm9ag0H6yXdETo7THPczZb')

        twitter_api = twitter.Twitter(auth=auth)
        
    movie=movie.replace(' ','')
    movie=movie.split(sep, 1)[0]
    search_results = twitter_api.search.tweets(q=movie,count=100)
    count=count+1
    statuses=json.dumps(search_results['statuses'])
    stat=json.loads(statuses)
    for s in stat:
        hashtags=''
        user_mentions=''
        for hashtag in s['entities']['hashtags']:
            hashtags=hashtags+' '+ hashtag['text']
        for user_mention in s['entities']['user_mentions']:
            user_mentions = user_mentions+' '+ user_mention['screen_name']
        tweets.loc[str(i)]=[movie,s['user']['id'],s['user']['screen_name'],s['id'],s['text'],hashtags,user_mentions,s['created_at'],s['retweet_count']] 
        i=i+1
        doc=doc+' '+s['text']
        doc=doc.strip().lower()
    movie_tweets.loc[str(movie_count)]=[movie,doc]
    movie_count=movie_count+1
#tweets
tweets.to_csv("Tweets.csv",encoding='utf-8')
movie_tweets.to_csv("Movie_Tweets.csv",encoding='utf-8')

Further store it into a databse and use NLTK to tokenize and extract tags using TF-IDF.


In [9]:
movies_df=pd.read_csv("imdb_movie_dataset.csv")

In [10]:
movies_df.head(3)

,certificate_id,certificate,director_id,director_name,genre_id,genre,gross_value,imdb_ratings,metscores,movie_description,Movie_id,movie_names,runtime,star_cast_id,star_cast,votes,year_release
0,2,PG-13,1350,Ryan Coogler,100,Action,291954422.0,7.9,88,"[""T'Challa, the King of Wakanda, rises to the ...",6.350540e+14,Black Panther,134,10676,Ryan Coogler,121117,2018
1,2,PG-13,1350,Ryan Coogler,104,Adventure,291954422.0,7.9,88,"[""T'Challa, the King of Wakanda, rises to the ...",6.350540e+14,Black Panther,134,10676,Ryan Coogler,121117,2018
2,2,PG-13,1350,Ryan Coogler,102,Sci,291954422.0,7.9,88,"[""T'Challa, the King of Wakanda, rises to the ...",6.350540e+14,Black Panther,134,10676,Ryan Coogler,121117,2018


In [11]:
from nltk.corpus import wordnet as wn
x=wn.synsets('film.n.01')

In [12]:
#loading the tweets
movie_tweets=pd.read_csv("Movie_Tweets.csv")
movie_tweets.head()

,movie_name,tweets
0,BlackPanther,just saw #blackpanther and now i keep randomly...
1,TheCloverfieldParadox,#nowwatching #thecloverfieldparadox en @netfli...
2,MazeRunner,@obrienupdatesuk @dylanobrien @kscodders @sang...
3,TheCommuter,rt @neesonaddict65: #liamneeson #thecommuter i...
4,FiftyShadesFreed,i saw '#fiftyshadesfreed' at @savoyheatonmoor....


# TF-IDF

We use the TF-IDF method to create tags based on their scores. The scoring is done by comparing the tokenized text with all the text for all the movies.
We compare each movie's text with every other movie- this way we get the most relevant words for each movie, rather than for movies in general.

In [15]:
# We create a list of blobs here.
bloblist=[]
for tweets in movie_tweets.itertuples():
    bloblist.append(tb(str(tweets[2])))
bloblist

[TextBlob("just saw #blackpanther and now i keep randomly yelling "wakanda for-evahhh!" hubs is not amused. rt @wakaflocka: we got him out!!! #danielkaluuya #london ðŸ‡¬ðŸ‡§ðŸ‡¬ðŸ‡§ðŸ‡¬ðŸ‡§ðŸ‡¬ðŸ‡§ðŸ‡¬ðŸ‡§ðŸ”¥ðŸ”¥ðŸ”¥ #blackpanther ðŸ’¨ðŸ’¨ðŸ’¨ tonite was insane!! @djwhookid ðŸ’¨ðŸ’¨ðŸ’¨ https://t.co/x8â€¦ rt @geeksofcolor: the dora milaje take center stage in new series from marvel comics #blackpanther https://t.co/xafrxiqs3r rt @rahelweldeab: real talk! 
 â€˜black pantherâ€™ and the anti-black racism of egyptians https://t.co/dsbuzwrrax #blackpanther #postcolonialissâ€¦ killmonger and shuri were the best character on #blackpanther 
 oh, and the rhino. 
 
 fight me, idc. the application of technology in #blackpanther is crazy inconsistent. even in a fictional universe where people canâ€¦ https://t.co/bpisilfqkb rt @theblackpanther: #blackpanther is the #1 movie in the world. see it again: https://t.co/nr2hefr37c (1/3) https://t.co/pkhgupptea rt @theblackpanther: #blackpanther is the #

In [2]:
import nltk
from __future__ import division
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')  + ['rt','RT','via']
import string
import math

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shruthisubbaiah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
from __future__ import division

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    n=0
    for blob in bloblist:
        if word in blob.words:
            n=n+1
    return n

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [18]:
movie_data=pd.read_csv('imdb_movie_dataset.csv')
movie_data.head(3)

,certificate_id,certificate,director_id,director_name,genre_id,genre,gross_value,imdb_ratings,metscores,movie_description,Movie_id,movie_names,runtime,star_cast_id,star_cast,votes,year_release
0,2,PG-13,1350,Ryan Coogler,100,Action,291954422.0,7.9,88,"[""T'Challa, the King of Wakanda, rises to the ...",6.350540e+14,Black Panther,134,10676,Ryan Coogler,121117,2018
1,2,PG-13,1350,Ryan Coogler,104,Adventure,291954422.0,7.9,88,"[""T'Challa, the King of Wakanda, rises to the ...",6.350540e+14,Black Panther,134,10676,Ryan Coogler,121117,2018
2,2,PG-13,1350,Ryan Coogler,102,Sci,291954422.0,7.9,88,"[""T'Challa, the King of Wakanda, rises to the ...",6.350540e+14,Black Panther,134,10676,Ryan Coogler,121117,2018


In [19]:
movie_ids=movie_data['Movie_id'].unique()
movie_ids

array([6.35054e+14, 1.07112e+15, 3.72742e+14, 1.41198e+14, 1.63800e+15,
       3.15079e+14, 5.19352e+14, 1.55367e+15, 1.17273e+15, 8.29155e+14,
       1.64459e+14, 2.09237e+14, 5.27837e+14, 3.12349e+14, 4.16379e+14,
       4.44921e+14, 3.14819e+14, 1.87816e+15, 1.30507e+15, 7.61458e+14,
       2.00684e+15, 8.06997e+14, 1.71519e+15, 8.78550e+14, 5.24958e+14,
       1.94895e+15, 1.91240e+14, 6.05136e+14, 1.27312e+15, 1.76587e+14,
       3.95334e+14, 1.26481e+14, 5.02393e+14, 3.05384e+14, 1.73003e+14])

In [20]:
movies=movie_data['movie_names'].unique()
len(movies)

35

In [21]:
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt','RT','via']

scores=pd.DataFrame(pd.DataFrame(columns=['movie_id','word','score']))
i=1
j=0
for blob in bloblist:
    for word in blob.words:
        if word not in stop and word.isalpha():
            scores.loc[str(i)]=[movie_ids[j],word,tfidf(word, blob, bloblist)]
            i=i+1
    j=j+1
scores

,movie_id,word,score
1,6.350540e+14,saw,0.003867
2,6.350540e+14,blackpanther,0.073960
3,6.350540e+14,keep,0.001083
4,6.350540e+14,randomly,0.001368
5,6.350540e+14,yelling,0.001594
6,6.350540e+14,wakanda,0.011156
7,6.350540e+14,hubs,0.001594
8,6.350540e+14,amused,0.001594
9,6.350540e+14,wakaflocka,0.001594
10,6.350540e+14,got,0.000935


In [22]:
scores=scores.sort_values(by=['score'],ascending=0)
scores=scores.drop_duplicates(keep='first')
scores=scores.groupby('movie_id').head(5).reset_index(drop=True)
scores.head()

,movie_id,word,score
0,1.730030e+14,nan,2.456736
1,1.264810e+14,nan,2.456736
2,3.727420e+14,mazerunnermovie,0.301284
3,3.727420e+14,dylanobrien,0.301284
4,3.727420e+14,sangsterthomas,0.301284


# Creating IMDB Movie Database

Our quest is to create a database for IMDB movies, such that we can save informations from various social media sources using this database schema. We have to make sure our database schema is flexible for any further addition to the list of tables.

This data will further be used with natural language processing for sentiment analysis.

In [23]:
#calling dependencies
import pandas as pd
import numpy as np
import sqlite3

## Datasets

The datasets we are using have been scraped (or API used) to gather the data set from various sources namely:

- [IMDB](http://www.imdb.com/)
- [facebook](https://www.facebook.com/)

The scripts for scraping the following can be found on my [GitHub](https://github.com/amar-chheda/Web-Scraping) profile.

In [140]:
#reading the movie dataset
movie_data = pd.read_csv("imdb_movie_dataset.csv")

In [141]:
#reading post dataset
post_data = pd.read_csv("post_data_fb.csv")
post_data = post_data.drop(['Unnamed: 0'], axis = 1)

In [142]:
#reading comment dataset
comment_data = pd.read_csv("comments_posts_fb.csv")
comment_data = comment_data.drop(['Unnamed: 0'], axis = 1)

In [143]:
#reading tweets database
tweets_data = pd.read_csv("Tweets.csv")

In [144]:
#reading the scores
tag_scores = pd.read_csv("scores.csv")
tag_scores = tag_scores[['movie_id', 'word', 'score']]

## Creating master tables abd normalizing the data into 3rd Normal form

We split our data into database tables to fit our database schema. 

In [145]:
#main table - All the other tables will be connected to this one
movie_master_table = pd.DataFrame()
movie_master_table[['movie_id','movie_names','movie_description','imdb_ratings','metascores','runtime','gross_value','year_release']] = movie_data[['Movie_id','movie_names','movie_description','imdb_ratings','metscores','runtime','gross_value','year_release']]
movie_master_table = movie_master_table.drop_duplicates()
movie_master_table.head(3)

,movie_id,movie_names,movie_description,imdb_ratings,metascores,runtime,gross_value,year_release
0,6.350540e+14,Black Panther,"[""T'Challa, the King of Wakanda, rises to the ...",7.9,88,134,291954422.0,2018
16,1.071120e+15,The Cloverfield Paradox,"['Orbiting a planet on the brink of war, scien...",5.7,37,102,NaN,2018
32,3.727420e+14,Maze Runner: The Death Cure,['Young hero Thomas embarks on a mission to fi...,6.8,51,141,55366604.0,2018


In [146]:
#starcast main table - this will connect to movie table using a seperate relational table
starcast_master_table = pd.DataFrame()
starcast_master_table[['starcast_id','starcast_name']] = movie_data[['star_cast_id','star_cast']]
starcast_master_table = starcast_master_table.drop_duplicates()
starcast_master_table.head(3)

,starcast_id,starcast_name
0,10676,Ryan Coogler
4,10677,Chadwick Boseman
8,10678,Michael B. Jordan


In [147]:
#director main table - this will connect to movie table using a seperate relational table
director_master_table = pd.DataFrame()
director_master_table[['director_id','director_name']] = movie_data[['director_id','director_name']]
director_master_table = director_master_table.drop_duplicates()
director_master_table.head(3)

,director_id,director_name
0,1350,Ryan Coogler
16,1351,Julius Onah
32,1352,Wes Ball


In [148]:
#genre main table - this will connect to movie table using a seperate relational table
genre_master_table = pd.DataFrame()
genre_master_table[['genre_id','genre']] = movie_data[['genre_id','genre']]
genre_master_table = genre_master_table.drop_duplicates()
genre_master_table.head(3)

,genre_id,genre
0,100,Action
1,104,Adventure
2,102,Sci


In [149]:
#posts main table - this will connect to movie table using a seperate relational table (these are posts from facebook)
posts_master_table = pd.DataFrame()
posts_master_table[['post_id','created_at','post_message','likes_count','share_count','comments_count','user_engagement']] = post_data[['post_id','created_at','post_message','post_likes_count','post_shares_count','post_comment_count','user_engagement']]
posts_master_table = posts_master_table.drop_duplicates()
posts_master_table['created_at'] = posts_master_table['created_at'].astype('datetime64[ns]')
posts_master_table.head(3)

,post_id,created_at,post_message,likes_count,share_count,comments_count,user_engagement
0,458711740828112_1857840717581867,2018-03-12 20:00:00,NaN,6,4.0,2,12
1,458711740828112_1858124704220135,2018-03-12 19:45:00,NaN,7,4.0,0,11
2,458711740828112_1857840970915175,2018-03-12 19:30:00,NaN,17,5.0,0,22


In [150]:
#comments main table - this will connect to posts table using a seperate relational table
comments_master_table = pd.DataFrame()
comments_master_table[['comments_id','created_at','comments']] = comment_data[['comments_id','created_at','post_comments']]
comments_master_table = comments_master_table.drop_duplicates()
comments_master_table['created_at'] = comments_master_table['created_at'].astype('datetime64[ns]')
comments_master_table.head(3)

,comments_id,created_at,comments
0,1857840717581867_1858341727531766,2018-03-12 20:06:32,NaN
1,1857840717581867_1858346307531308,2018-03-12 20:11:39,"Vero grazie, buonanotte"
2,1857951664237439_1858307817535157,2018-03-12 19:21:25,Mia


In [151]:
#tweets main table - this will connect the tweets to the movies
tweets_master_table = pd.DataFrame()
tweets_master_table[['tweet_id','tweet_text', 'created_date', 'retweet_count']] = tweets_data[['tweet_id','tweet_text','created_date','retweet_count']]
tweets_master_table = tweets_master_table.drop_duplicates()
tweets_master_table.head(3)

,tweet_id,tweet_text,created_date,retweet_count
0,1111,Just saw #BlackPanther and now I keep randomly...,Sat Mar 17 20:41:14 +0000 2018,0
1,1112,RT @WakaFlocka: We Got Him OUT!!! #DanielKaluu...,Sat Mar 17 20:41:13 +0000 2018,96
2,1113,RT @GeeksOfColor: The Dora Milaje Take Center ...,Sat Mar 17 20:41:00 +0000 2018,281


In [152]:
#twitter user table - this links the user with tweets
twitter_user_table = pd.DataFrame()
twitter_user_table[['user_id','user_name']] = tweets_data[['user_id','screen_name']]
twitter_user_table = twitter_user_table.drop_duplicates()
twitter_user_table.head(3)

,user_id,user_name
0,121211,happyhealthyacw
1,121212,ClintonS_anchez
2,121213,blackaqualad


## Creating Mappings between all the tables

Here, we create seperate mapping tables between the database tables in order to perform join statements.


In [153]:
#movie-director mapping
movie_director_maping = pd.DataFrame()
movie_director_maping[['movie_id','director_id']] = movie_data[['Movie_id','director_id']]
movie_director_maping = movie_director_maping.drop_duplicates()
movie_director_maping.head(3)

,movie_id,director_id
0,6.350540e+14,1350
16,1.071120e+15,1351
32,3.727420e+14,1352


In [154]:
#movie-starcast mapping
movie_starcast_maping = pd.DataFrame()
movie_starcast_maping[['movie_id','starcast_id']] = movie_data[['Movie_id','star_cast_id']]
movie_starcast_maping = movie_starcast_maping.drop_duplicates()
movie_starcast_maping.head(3)

,movie_id,starcast_id
0,6.350540e+14,10676
4,6.350540e+14,10677
8,6.350540e+14,10678


In [155]:
#movie-genre mapping
movie_genre_maping = pd.DataFrame()
movie_genre_maping[['movie_id','genre_id']] = movie_data[['Movie_id','genre_id']]
movie_genre_maping = movie_genre_maping.drop_duplicates()
movie_genre_maping.head(3)

,movie_id,genre_id
0,6.350540e+14,100
1,6.350540e+14,104
2,6.350540e+14,102


In [156]:
#movie-post mapping
movie_post_maping = pd.DataFrame()
movie_post_maping[['movie_id','post_id']] = post_data[['movie_id','post_id']]
movie_post_maping = movie_post_maping.drop_duplicates()
movie_post_maping.head(3)

,movie_id,post_id
0,6.350540e+14,458711740828112_1857840717581867
1,6.350540e+14,458711740828112_1858124704220135
2,6.350540e+14,458711740828112_1857840970915175


In [157]:
#post-comment mapping
post_comment_maping = pd.DataFrame()
post_comment_maping[['post_id','comments_id']] = comment_data[['post_id','comments_id']]
post_comment_maping = post_comment_maping.drop_duplicates()
post_comment_maping.head(3)

,post_id,comments_id
0,458711740828112_1857840717581867,1857840717581867_1858341727531766
1,458711740828112_1857840717581867,1857840717581867_1858346307531308
2,458711740828112_1857951664237439,1857951664237439_1858307817535157


In [158]:
#movie-tweets mapping
movie_tweets_mapping = pd.DataFrame()
movie_tweets_mapping[['movie_id','tweet_id']] = tweets_data[['movie_id','tweet_id']]
movie_tweets_mapping = movie_tweets_mapping.drop_duplicates()
movie_tweets_mapping.head(3)

,movie_id,tweet_id
0,6.350540e+14,1111
1,6.350540e+14,1112
2,6.350540e+14,1113


In [159]:
#tweets-user mapping
tweets_user_mapping = pd.DataFrame()
tweets_user_mapping[['tweet_id', 'user_id']] = tweets_data[['tweet_id','user_id']]
tweets_user_mapping = tweets_user_mapping.drop_duplicates()
tweets_user_mapping.head(3)

,tweet_id,user_id
0,1111,121211
1,1112,121212
2,1113,121213


## Creating databases

Now that we have our schma and the tables for the database ready, let us start creating the tables in the databases and start storing the vales in them.

In [160]:
#creating connections
conn = sqlite3.connect("imdb_movie.db") #creates and connects to a database named "imdb_movie"
c = conn.cursor()

In [161]:
c.execute("""DROP TABLE movie_master_table""")
c.execute("""CREATE TABLE movie_master_table(
movie_id INTEGER PRIMARY KEY,
movie_names CHAR(50),
movie_description VARCHAR(200),
imdb_ratings FLOAT,
metascores INTEGER,
runtime INTEGER,
gross_value INTEGER,
year_release CHAR(4));""") #creating a new tables within database

In [162]:
movie_master_table.to_sql("movie_master_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [163]:
c.execute("""DROP TABLE starcast_master_table;""")
c.execute("""CREATE TABLE starcast_master_table(
starcast_id INTEGER PRIMARY KEY,
starcast_name CHAR NOT NULL);""") #creating a new tables within database

In [164]:
starcast_master_table.to_sql("starcast_master_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [165]:
c.execute("""DROP TABLE director_master_table;""")
c.execute("""CREATE TABLE director_master_table(
director_id INTEGER PRIMARY KEY,
director_name CHAR NOT NULL);""") #creating a new tables within database

In [166]:
director_master_table.to_sql("director_master_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [167]:
c.execute("""DROP TABLE genre_master_table;""")
c.execute("""CREATE TABLE genre_master_table(
genre_id INTEGER PRIMARY KEY,
genre CHAR NOT NULL);""") #creating a new tables within database

In [168]:
genre_master_table.to_sql("genre_master_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [169]:
c.execute("""DROP TABLE posts_master_table;""")
c.execute("""CREATE TABLE posts_master_table(
post_id CHAR PRIMARY KEY,
created_at DATE,
post_message CHAR,
likes_count INTEGER,
share_count INTEGER,
comments_count INTEGER,
user_engagement INTEGER) ;""") #creating a new tables within database

In [170]:
posts_master_table.to_sql("posts_master_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [171]:
c.execute("""DROP TABLE comments_master_table;""")
c.execute("""CREATE TABLE comments_master_table(
comments_id CHAR PRIMARY KEY,
created_at DATE,
comments CHAR);""") #creating a new tables within database

In [172]:
comments_master_table.to_sql("comments_master_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [173]:
c.execute("""DROP TABLE tweets_master_table;""")
c.execute("""CREATE TABLE tweets_master_table(
tweet_id INTEGER PRIMARY KEY,
tweet_text CHAR,
created_date DATE,
retweet_count INTEGER);""") #creating a new tables within database

In [174]:
tweets_master_table.to_sql("tweets_master_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [175]:
c.execute("""DROP TABLE twitter_user_table;""")
c.execute("""CREATE TABLE twitter_user_table(
user_id INTEGER PRIMARY KEY,
user_name CHAR);""") #creating a new tables within database

In [176]:
twitter_user_table.to_sql("twitter_user_table", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [177]:
c.execute("""DROP TABLE movie_tags;""")
c.execute("""CREATE TABLE movie_tags(
movie_id INTEGER FOREIOGN KEY,
word CHAR,
score FLOAT);""") #creating a new table within database

In [178]:
tag_scores.to_sql("movie_tags", conn, if_exists = "append", index = False) #storing dataframe to SQL database

## Creating connection to the database

Here, we are using SQLite database management system in order to create store and analyse our data. we create a relational database schema and store our data inside.


In [179]:
c.execute("""DROP TABLE movie_director_maping;""")
c.execute("""CREATE TABLE movie_director_maping(
movie_id INTEGER FOREGION KEY,
director_id INTEGER FOREGION KEY)""") #creating a new tables within database

In [180]:
movie_director_maping.to_sql("movie_director_maping", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [181]:
c.execute("""DROP TABLE movie_starcast_maping;""")
c.execute("""CREATE TABLE movie_starcast_maping(
movie_id INTEGER FOREGION KEY,
starcast_id INTEGER FOREGION KEY)""") #creating a new tables within database

In [182]:
movie_starcast_maping.to_sql("movie_starcast_maping", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [183]:
c.execute("""DROP TABLE movie_genre_maping;""")
c.execute("""CREATE TABLE movie_genre_maping(
movie_id INTEGER FOREGION KEY,
genre_id INTEGER FOREGION KEY)""") #creating a new tables within database

In [184]:
movie_genre_maping.to_sql("movie_genre_maping", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [185]:
c.execute("""DROP TABLE movie_post_maping;""")
c.execute("""CREATE TABLE movie_post_maping(
movie_id INTEGER FOREGION KEY,
post_id CHAR FOREGION KEY)""") #creating a new tables within database

In [186]:
movie_post_maping.to_sql("movie_post_maping", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [187]:
c.execute("""DROP TABLE post_comment_maping;""")
c.execute("""CREATE TABLE post_comment_maping(
post_id CHAR FOREGION KEY,
comments_id CHAR FOREGION KEY)""") #creating a new tables within database

In [188]:
post_comment_maping.to_sql("post_comment_maping", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [189]:
c.execute("""DROP TABLE movie_tweets_mapping;""")
c.execute("""CREATE TABLE movie_tweets_mapping(
movie_id CHAR FOREGION KEY,
tweet_id CHAR FOREGION KEY)""") #creating a new tables within database

In [190]:
movie_tweets_mapping.to_sql("movie_tweets_mapping", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

In [191]:
c.execute("""DROP TABLE tweets_user_mapping;""")
c.execute("""CREATE TABLE tweets_user_mapping(
tweet_id CHAR FOREGION KEY,
user_id CHAR FOREGION KEY)""") #creating a new tables within database

In [192]:
tweets_user_mapping.to_sql("tweets_user_mapping", conn, if_exists = 'append', index = False) #Storing data frame to SQL database

## Answering Questions

Now that we have aggregated our dataset, let us go ahead and try to answer certain basic questions on the basis of data we have collected. 

I will answer all this questions by implementing our usecases using SQL queries 

In [3]:
conn = sqlite3.connect("imdb_movie.db") #creates and connects to a database named "imdb_movie"
c = conn.cursor()

##### Before we start answering any questions, "me" or the person of interest in our case is an official movie_page. Any questions we are answering is with the notion of movie_page as my user of interest

# Question 1:
## What are people saying about me?
We answer this question by looking at the comments on our posts, this tells us the overall sentiment people have regarding a particular movie. 

In [15]:
pd.read_sql_query("""SELECT c.comments from 
movie_master_table as m INNER JOIN movie_post_maping as mp ON m.movie_id=mp.movie_id
INNER JOIN posts_master_table as p ON p.post_id=mp.post_id
INNER JOIN post_comment_maping as pc ON p.post_id=pc.post_id
INNER JOIN comments_master_table as c ON pc.comments_id=c.comments_id
WHERE m.movie_names='Black Panther' and c.comments NOT NULL
""",conn)

,comments
0,"Vero grazie, buonanotte"
1,Mia
2,Verissimo .
3,"Soffro ma tenace,,"
4,e speriamo che sia una bella persona questo al...
5,Mai nessuno lo farà 👍👍
6,presa
7,Io ce convivo coi perché. ...e so perché che f...
8,💖💖🐾🦁🌻😘
9,Esatto 👍👏


# Question 2:

## How viral are my posts?

For answering this question, we take a weighted sum of the count of likes, shares and comments on my posts. The maximum weightage is given to the share count as that signifies the max spreading of any post, followed by the likes and comments.

In [26]:
pd.read_sql_query(""" SELECT movie_names, SUM((0.5*p.share_count)+(0.3*p.likes_count)+(0.2*p.comments_count)) as viral
FROM movie_master_table m
JOIN movie_post_maping mp ON m.movie_id = mp.movie_id
JOIN posts_master_table p ON p.post_id = mp.post_id
GROUP BY m.movie_names
ORDER BY viral DESC LIMIT 7;""", conn)

,movie_names,viral
0,Black Panther,41265.9
1,12 Strong,36367.8
2,Den of Thieves,26541.8
3,Winchester,22367.1
4,Padmaavat,18778.1
5,Fifty Shades Freed,9983.6
6,Maze Runner: The Death Cure,9588.6


# Question 3:

## How much influence do my posts have?

To answer the question we check for shares of the post being posted by the movie page.

In [29]:
pd.read_sql_query(""" SELECT movie_names, SUM(retweet_count) as Influence
FROM movie_master_table as m
INNER JOIN movie_tweets_mapping mt ON mt.movie_id = m.movie_id
INNER JOIN tweets_master_table t ON mt.tweet_id = t.tweet_id
GROUP BY m.movie_names
ORDER BY Influence DESC LIMIT 7;""", conn)

,movie_names,Influence
0,Fifty Shades Freed,455501
1,Every Day,190621
2,Black Panther,158901
3,Maze Runner: The Death Cure,116082
4,Braven,106952
5,Mandy,96149
6,Den of Thieves,87488


# Question 4:

## What posts are like mine?

For this, we considerd the posts of two users- DAMSEL_011 and Annyseventwo. One of this can refer to 'Me'. And then checked if they had common words other than stopwords. If yes, their posts are similar


In [104]:
user1_posts=pd.read_sql("""SELECT u.user_name, t.tweet_text
FROM tweets_master_table as t
INNER JOIN tweets_user_mapping as tu ON t.tweet_id=tu.tweet_id
INNER JOIN twitter_user_table as u ON tu.user_id=u.user_id
WHERE u.user_name='DAMSEL_011'
""",conn)

user2_posts=pd.read_sql("""SELECT u.user_name, t.tweet_text
FROM tweets_master_table as t
INNER JOIN tweets_user_mapping as tu ON t.tweet_id=tu.tweet_id
INNER JOIN twitter_user_table as u ON tu.user_id=u.user_id
WHERE u.user_name='Annyseventwo'
""",con=conn)

In [105]:
from nltk.corpus import stopwords
stop = stopwords.words('english') + ['rt','RT','via']
user1_tags=user1_posts.tweet_text.unique()
user2_tags=user2_posts.tweet_text.unique()
common_tags=''
flag=0
common_count=0
for item in user1_tags:
    for i in item.lower().split(" "):
        for item2 in user2_tags:
            for i2 in item2.lower().split(" "):
                if i not in stop and i.isalpha():
                    if i in i2 and i not in common_tags:
                        common_count=common_count+1
                        common_tags=common_tags+i+','       
if common_count==0:
    print "They do not have anything in common"
else:
    print "They are similar, they have posted about",common_tags

They are similar, they have posted about wish,e,watch,day,really,


# Question 5:
## What users post like me?

For this, we considerd the posts of two users- DAMSEL_011 and Annyseventwo. And then checked if they had common words other than stopwords. If yes, their posts are similar

In [66]:
user1_posts=pd.read_sql("""SELECT u.user_name, t.tweet_text
FROM tweets_master_table as t
INNER JOIN tweets_user_mapping as tu ON t.tweet_id=tu.tweet_id
INNER JOIN twitter_user_table as u ON tu.user_id=u.user_id
WHERE u.user_name='DAMSEL_011'
""",conn)

user2_posts=pd.read_sql("""SELECT u.user_name, t.tweet_text
FROM tweets_master_table as t
INNER JOIN tweets_user_mapping as tu ON t.tweet_id=tu.tweet_id
INNER JOIN twitter_user_table as u ON tu.user_id=u.user_id
WHERE u.user_name='Annyseventwo'
""",con=conn)


In [67]:
user2_posts

,user_name,tweet_text
0,Annyseventwo,RT @ronaldtheshort: Damsel was fantastic! I’ve...
1,Annyseventwo,"RT @FilmATX: #SXSW: Damsel, the new film from ..."
2,Annyseventwo,RT @andrewetn: Now seating for the Zellner Bro...
3,Annyseventwo,RT @cheektogeek: Last day of #sxsw. Seated for...


In [68]:
user1_posts

,user_name,tweet_text
0,DAMSEL_011,@Mzlianne 🤣🤣🤣🤣
1,DAMSEL_011,"RT @zenclaire29: This guy is a hunk though, so..."
2,DAMSEL_011,@jenny2209tisia I wish she was around o cos if...
3,DAMSEL_011,Lo tori mi loje gbese e #bbnaija
4,DAMSEL_011,Dj enjoying his handwork 😁 #bbnaija
5,DAMSEL_011,I better start recording moments. I need to be...
6,DAMSEL_011,Wow Rico! Song sounds really good. Awww #bbnaija
7,DAMSEL_011,"MINA must get married in this outfit, Sheik To..."
8,DAMSEL_011,@ekiloui Absolute rubbish.
9,DAMSEL_011,@TeamTobiforever At all


In [84]:
from nltk.corpus import stopwords
stop = stopwords.words('english') + ['rt','RT','via']

In [102]:
user1_tags=user1_posts.tweet_text.unique()
user2_tags=user2_posts.tweet_text.unique()
common_tags=''
flag=0
common_count=0
for item in user1_tags:
    for i in item.lower().split(" "):
        for item2 in user2_tags:
            for i2 in item2.lower().split(" "):
                if i not in stop and i.isalpha():
                    if i in i2 and i not in common_tags:
                        common_count=common_count+1
                        common_tags=common_tags+i+','       
if common_count==0:
    print "They do not have anything in common"
else:
    print "They are similar, they have posted about",common_tags
       

They are similar, they have posted about wish,e,watch,day,really,


# Question 6:
## Who should I be following?

For answering this question we calculate the number of max retweets for posts by a user. We chose the user because he brings in the maximum engagement for the posts related to our domain and following him back would pull in movie buffs who follow him and this would benefit us.



In [12]:
pd.read_sql_query(""" SELECT u.user_name, SUM(retweet_count) as Total_retweets
FROM  movie_master_table as m 
INNER JOIN movie_tweets_mapping as mt ON m.movie_id=mt.movie_id
INNER JOIN tweets_master_table as t ON t.tweet_id=mt.tweet_id
INNER JOIN tweets_user_mapping as ut ON ut.tweet_id=t.tweet_id
INNER JOIN twitter_user_table as u ON ut.user_id=u.user_id
GROUP BY u.user_id
ORDER BY Total_retweets desc LIMIT 7
""",conn)

,user_name,Total_retweets
0,mokgadi_mandy,65128
1,mlsfrs,64235
2,Braven_Kerbs,60995
3,1975Peyton,30341
4,nana__yd,30341
5,____hudax,30341
6,Braven_Kerbs,29856


# Question 7:

## What topics are trending between 10th and 12th March 2018??

We are answering this question using the most famous tags. We want to get the tags we have in an descending order for this purpose.

In [33]:
pd.read_sql_query("""SELECT movie_names,word,score
FROM movie_master_table as m INNER JOIN 
movie_tags as mt ON mt.movie_id=m.movie_id and word NOT NULL and 
ORDER BY mt.score DESC LIMIT 7
""",conn)    

,movie_names,word,score
0,Maze Runner: The Death Cure,mazerunnermovie,0.301284
1,Maze Runner: The Death Cure,dylanobrien,0.301284
2,Maze Runner: The Death Cure,sangsterthomas,0.301284
3,Kickboxer: Retaliation,kickboxer,0.173467
4,Damsel,sinazomanazo,0.168365
5,Damsel,damsel,0.168365
6,Scorched Earth,scorchedearth,0.168365


# Question 8:

## What keywords / Hashtags Should I add to my post?

We are answering this question using the most famous tags for each movie. 


In [35]:
pd.read_sql_query("""SELECT movie_names, group_concat(word) 
FROM movie_tags as mt, movie_master_table as mmt 
WHERE mmt.movie_id == mt.movie_id GROUP BY movie_names""",conn)

,movie_names,group_concat(word)
0,12 Strong,"passes,carload,starlitewichita,fri,open"
1,A Futile and Stupid Gesture,"afutileandstupidgesture,peterprincipato,mikeco..."
2,Acts of Violence,"actsofviolence,brucewillis,therealmikeepps,tra..."
3,Annihilation,"annihilation,bears,dmeishappy,paddington,netflix"
4,Black Panther,"blackpanther,theblackpanther,marvelstudios,wor..."
5,Braven,"braven,umbc,towson,beat,uva"
6,Damsel,"sinazomanazo,damsel,yoh,peacehochub,creating"
7,Den of Thieves,"denofthieves,l,gerardbutler,outlaws,power"
8,"Don't Worry, He Won't Get Far on Foot",None
9,Early Man,"earlyman,aardman,thesimpleparent,visa,gc"


# Question 9:
## Should I follow somebody back?
For this question, we are going to see if a user is posting about similar topics as me(user1). If yes, follow the 
user back because we have similar interests.


In [106]:
user1_posts=pd.read_sql("""SELECT u.user_name, t.tweet_text
FROM tweets_master_table as t
INNER JOIN tweets_user_mapping as tu ON t.tweet_id=tu.tweet_id
INNER JOIN twitter_user_table as u ON tu.user_id=u.user_id
WHERE u.user_name='DAMSEL_011'
""",conn)

user2_posts=pd.read_sql("""SELECT u.user_name, t.tweet_text
FROM tweets_master_table as t
INNER JOIN tweets_user_mapping as tu ON t.tweet_id=tu.tweet_id
INNER JOIN twitter_user_table as u ON tu.user_id=u.user_id
WHERE u.user_name='Annyseventwo'
""",con=conn)

In [107]:
user1_tags=user1_posts.tweet_text.unique()
user2_tags=user2_posts.tweet_text.unique()
common_tags=''
flag=0
common_count=0
for item in user1_tags:
    for i in item.lower().split(" "):
        for item2 in user2_tags:
            for i2 in item2.lower().split(" "):
                if i not in stop and i.isalpha():
                    if i in i2 and i not in common_tags:
                        common_count=common_count+1
                        common_tags=common_tags+i+','       
if common_count==0:
    print "They do not have anything in common"
else:
    print "They are similar, they have posted about",common_tags

They are similar, they have posted about wish,e,watch,day,really,


# Question 10:
## What is the best time to post?

To answer this question we see that Maximum count of retweets and see the time stamp for that post. Posting at that time would bring us maximum user engagement.

In [39]:
pd.read_sql_query("""SELECT SUM(retweet_count) as Retweets,t.created_date
FROM movie_master_table as m 
INNER JOIN movie_tweets_mapping as mt ON m.movie_id=mt.movie_id
INNER JOIN tweets_master_table as t ON t.tweet_id=mt.tweet_id
GROUP BY t.created_date
ORDER BY Retweets DESC LIMIT 20
""",conn)

,Retweets,created_date
0,65128,Sat Mar 17 20:36:44 +0000 2018
1,64235,Sat Mar 17 20:34:21 +0000 2018
2,60995,Sat Mar 17 03:35:39 +0000 2018
3,30344,Sat Mar 17 20:38:24 +0000 2018
4,30341,Sat Mar 17 20:34:39 +0000 2018
5,30341,Sat Mar 17 20:36:03 +0000 2018
6,29856,Sat Mar 17 02:47:42 +0000 2018
7,27796,Sat Mar 17 16:57:21 +0000 2018
8,27796,Sat Mar 17 17:01:51 +0000 2018
9,27796,Sat Mar 17 17:35:31 +0000 2018


# Question 11:
## Should I add a picture / URL to my posts?

Do the most retweeted posts have a Picture/ URL?

We select the tweets with most number of re-tweets and check if it contains a URL or Picture, if "Yes" we add it to our post if "No" we do not add anything to our post.

In [52]:
pd.read_sql_query("""SELECT DISTINCT m.movie_names, tweet_text , 
CASE WHEN tweet_text LIKE '%https%' or tweet_text LIKE '%http%'
       THEN 'Yes'
       ELSE 'No'
       END AS Pic_UrlPresent 
FROM movie_master_table as m 
INNER JOIN movie_tweets_mapping as mt ON m.movie_id=mt.movie_id
INNER JOIN tweets_master_table as t ON t.tweet_id=mt.tweet_id
ORDER BY retweet_count DESC LIMIT 30
""",conn)

,movie_names,tweet_text,Pic_UrlPresent
0,Mandy,RT @UglyGod: Ugly people RT.,No
1,Game Night,RT @BT21_: Who’s the winner? #gamenight #weeke...,Yes
2,Braven,RT @UMBCAthletics: PUT SOME RESPECK ON IT! WE ...,No
3,Black Panther,RT @Lupita_Nyongo: When your prom date is lite...,Yes
4,Braven,RT @LakeShowFamily: This is a cursed picture o...,Yes
5,Fifty Shades Freed,RT @LiamPayne: Here's the #ForYou (#FiftyShade...,Yes
6,Maze Runner: The Death Cure,RT @SangsterThomas: The voice of reason: Newt!...,Yes
7,Fifty Shades Freed,RT @LiamPayne: #FiftyShadesFreed Premiere toni...,Yes
8,Every Day,RT @iconnnz: ไม่ได้ตามดราม่า แต่ถ้าถามว่าเสื้อ...,No
9,The Mercy,RT @tofakeit: Thread of my favorite moments in...,No


# Question 12:
## Whats my reach?

For this question, I calculated the mean retweets that a user gets for all his tweets.
I tried to compare this to the number of followers he has, to get an estimate of how many people outside his
circle have shared his tweet. However, the Twitter API for the followers list just returns the 20 first followers of the user- there is no way to determine his total followers count.



In [157]:
i=1
user_id=620457220

followers_df=pd.DataFrame(columns=['user_id','friend_id'])
follow_list=twitter_api.followers.list(user_id =620457220 )
friends=json.dumps(follow_list['users'])
fr=json.loads(friends)
for f in fr:
    followers_df.loc[str(i)]= [user_id,f['id']]
    i=i+1
len(followers_df)

20

In [168]:
pd.read_sql_query("""SELECT u.user_name, AVG(t.retweet_count) as Mean_retweets
FROM tweets_master_table as t
INNER JOIN tweets_user_mapping as ut ON ut.tweet_id=t.tweet_id
INNER JOIN twitter_user_table as u ON ut.user_id=u.user_id
GROUP BY u.user_name
ORDER BY Mean_retweets DESC
""",conn)

,user_name,Mean_retweets
0,mokgadi_mandy,65128.000
1,mlsfrs,64235.000
2,1975Peyton,30341.000
3,____hudax,30341.000
4,nana__yd,30341.000
5,Escarleth230710,27796.000
6,JarelyGonzalez,27796.000
7,Niallisunique,27796.000
8,Pedro0631704997,27796.000
9,Saezmaikelly,27796.000
